In [ ]:
#include <Servo.h>

// Create an instance of the Servo library
Servo myServo;

// Pin definitions
const int piezoPin = A0;
const int switchPin = 2;
const int yellowLed = 3;
const int greenLed = 4;
const int redLed = 5;

// Knock value thresholds
const int quietKnock = 10;
const int loudKnock = 100;

// State variables
bool locked = false;
int numberOfKnocks = 0;

void setup() {
    // Attach the servo to pin 9
    myServo.attach(9);
    
    // Set LED pins as outputs
    pinMode(yellowLed, OUTPUT);
    pinMode(redLed, OUTPUT);
    pinMode(greenLed, OUTPUT);
    
    // Set the switch pin as an input
    pinMode(switchPin, INPUT);
    
    // Start serial communication for debugging
    Serial.begin(9600);
    
    // Initial state: unlocked
    digitalWrite(greenLed, HIGH);
    myServo.write(0);
    Serial.println("The box is unlocked!");
}

void loop() {
    // If the box is unlocked
    if (!locked) {
        if (digitalRead(switchPin) == HIGH) {
            // Lock the box with debounce
            delay(50); // Simple debounce delay
            if (digitalRead(switchPin) == HIGH) {
                locked = true;
                digitalWrite(greenLed, LOW);
                digitalWrite(redLed, HIGH);
                myServo.write(90);
                Serial.println("The box is locked!");
                delay(100);
            }
        }
    } 
    // If the box is locked
    else {
        int knockVal = analogRead(piezoPin);
        
        if (knockVal > 0) {
            if (checkForKnock(knockVal)) {
                numberOfKnocks++;
                Serial.print(3 - numberOfKnocks);
                Serial.println(" more knocks to go");
                delay(500); // Simple delay to avoid counting the same knock
            }
        }

        // Unlock the box after 3 valid knocks
        if (numberOfKnocks >= 3) {
            locked = false;
            numberOfKnocks = 0; // Reset knock count
            myServo.write(0);
            delay(10);
            digitalWrite(greenLed, HIGH);
            digitalWrite(redLed, LOW);
            Serial.println("The box is unlocked!");
        }
    }
}


// Function to check if the knock is within the valid range
bool checkForKnock(int value) {
    if (value > quietKnock && value < loudKnock) {
        digitalWrite(yellowLed, HIGH);
        delay(10);
        digitalWrite(yellowLed, LOW);
        Serial.print("Valid knock of value ");
        Serial.println(value);
        return true;
    } else {
        Serial.print("Bad knock value ");
        Serial.println(value);
        return false;
    }
}